In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
from minsearch import AppendableIndex

index = AppendableIndex(   #same as index , except we can add documents into the index
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [4]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True        
            )

    return results

In [5]:
question = 'can i still join the course?'

In [6]:
def build_prompt(query, search_results):
    context = "" 
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    print(context)
    prompt = f""" 
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the question. If the CONTEXT doesn't contain the answer, output NONE

    <QUESTION>{query}</QUESTION>
    <CONTEXT>{context}</CONTEXT>
    """.strip()

    return prompt



In [7]:
from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [8]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [9]:
rag(question)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Certificate - Can I follow the course in a self-paced mode and get a certificate?
answer: No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You c

"Yes, you can still join the course after the start date. Even if you don't register, you're still eligible to submit the homeworks, but be aware of deadlines for final projects."

### Agentic RAG

In [10]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [11]:
question = 'Can I still join the course?'
context = 'EMPTY'

In [12]:
prompt = prompt_template.format(question=question, context=context)

In [13]:
answer_json = llm(prompt)

In [14]:
import json

answer = json.loads(answer_json)

In [15]:
answer

{'action': 'SEARCH',
 'reasoning': 'The question asks about the possibility of joining the course, which is not addressed in the current context. Hence, I will refer to the FAQ database to find relevant information.'}

In [16]:

def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    return context.strip()

In [17]:
search_results = search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)

In [18]:
answer_json = llm(prompt)
answer_json

'{\n"action": "ANSWER",\n"answer": "Yes, you can still join the course after the start date. Even if you haven\'t registered, you are still able to submit homework assignments. Just keep in mind that there are deadlines for the final projects, so it\'s important to manage your time effectively and not procrastinate.",\n"source": "CONTEXT"\n}'

In [19]:
print(answer_json)

{
"action": "ANSWER",
"answer": "Yes, you can still join the course after the start date. Even if you haven't registered, you are still able to submit homework assignments. Just keep in mind that there are deadlines for the final projects, so it's important to manage your time effectively and not procrastinate.",
"source": "CONTEXT"
}


## Agentic Search

In [20]:
def dedup(seq):
    seen = set()
    result = []
    for el in seq:
        _id = el['_id']
        if _id in seen:
            continue
        seen.add(_id)
        result.append(el)
    return result

In [21]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

Output templates:

If you want to perform search, use this template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [22]:
question = 'how do I do well on module 1'
max_iterations = 3
iteration_number = 0
search_queries = []
search_results  = []
previous_actions = []

In [23]:
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)


In [24]:
answer_json = llm(prompt)

In [25]:
answer = json.loads(answer_json)

In [26]:
print(answer_json)

{
"action": "SEARCH",
"reasoning": "There are currently no details about module 1 in the context, so I will search for best practices or tips for succeeding in the first module of the course.",
"keywords": ["tips for success module 1", "how to do well in first module", "study techniques module 1"]
}


In [27]:

previous_actions.append(answer)

In [28]:

keywords = answer['keywords']

In [29]:

for kw in keywords:
    search_queries.append(kw)
    sr = search(kw)
    search_results.extend(sr)

In [30]:

search_results = dedup(search_results)

In [31]:
iteration_number = 2

context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [32]:
answer_json = llm(prompt)

In [33]:
print(answer_json)

{
"action": "SEARCH",
"reasoning": "I need to find specific tips or resources that can help students succeed in Module 1, focusing on Docker and Terraform, as there is still no relevant context for this module.",
"keywords": ["successful strategies for Docker and Terraform", "best practices for Module 1", "Docker Terraform study tips"]
}


In [34]:
question = "what do I need to do to be successful at module 1?"

search_queries = []
search_results = []
previous_actions = []

iteration = 0

while True:
    print(f'ITERATION #{iteration}...')

    context = build_context(search_results)
    prompt = prompt_template.format(
        question=question,
        context=context,
        search_queries="\n".join(search_queries),
        previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
        max_iterations=3,
        iteration_number=iteration
    )

    print(prompt)

    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(json.dumps(answer, indent=2))

    previous_actions.append(answer)

    action = answer['action']
    if action != 'SEARCH':
        break

    keywords = answer['keywords']
    search_queries = list(set(search_queries) | set(keywords))
    
    for k in keywords:
        res = search(k)
        search_results.extend(res)

    search_results = dedup(search_results)
    
    iteration = iteration + 1
    if iteration >= 4:
        break

    print()

ITERATION #0...
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current 

In [35]:
answer

{'action': 'ANSWER',
 'answer': "To be successful at Module 1, which focuses on Docker and Terraform, it's important to follow these strategies:\n\n1. **Understand the Basics**: Make sure you grasp the core concepts of Docker and Terraform. Spend some time on their official documentation to familiarize yourself with their functionalities and architecture.\n\n2. **Hands-On Practice**: Set up a local environment where you can practice using Docker to containerize applications and utilize Terraform for infrastructure as code. Engage in hands-on projects to reinforce learning.\n\n3. **Common Issues and Solutions**: Be aware of common issues that students face, such as module dependencies (like `psycopg2` for Postgres), and how to troubleshoot them. Use the community forums or resources for guidance on resolving these challenges.\n\n4. **Network Configuration Skills**: Ensure that you have a solid understanding of network settings and how they affect your Docker containers and Terraform set

## Function calling ("tool use")

In [36]:

def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [37]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [38]:

def do_call(tool_call_response):
    function_name = tool_call_response.name
    arguments = json.loads(tool_call_response.arguments)

    f = globals()[function_name]
    result = f(**arguments)

    return {
        "type": "function_call_output",
        "call_id": tool_call_response.call_id,
        "output": json.dumps(result, indent=2),
    }

In [39]:
question = "How do I do well in module 1?"

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
If you look up something in FAQ, convert the student question into multiple queries.
""".strip()

tools = [search_tool]

chat_messages = [
    {"role": "developer", "content": developer_prompt},
    {"role": "user", "content": question}
]

response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseFunctionToolCall(arguments='{"query":"how to do well in module 1"}', call_id='call_Ba9DFlwFzHH8MmokmrSCHcfT', name='search', type='function_call', id='fc_6878626f62c88198a0edaf7084a18f3106ca1fb88b5fdcc8', status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"tips for success in module 1"}', call_id='call_JPS6cHjI8tMWPEuX50L0aBuM', name='search', type='function_call', id='fc_6878626fa14c8198a7e2ca3d2fbfe6ac06ca1fb88b5fdcc8', status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"study strategies module 1"}', call_id='call_PUIY4cy7frbCbRV3DNH58pu0', name='search', type='function_call', id='fc_6878626febb481989c1f9b989317a9d006ca1fb88b5fdcc8', status='completed')]

In [40]:

calls = response.output

In [41]:

for call in calls:
    result = do_call(call)
    chat_messages.append(call)
    chat_messages.append(result)

In [42]:
response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseOutputMessage(id='msg_6878628668a48198b4e1ec55dc62c89806ca1fb88b5fdcc8', content=[ResponseOutputText(annotations=[], text="To excel in Module 1 of the course, consider the following strategies and tips:\n\n1. **Understand the Core Concepts**:\n   - Focus on the principles of Docker and Terraform since Module 1 primarily covers these technologies. Review any foundational materials provided in the course.\n\n2. **Hands-On Practice**:\n   - Set up a local environment using Docker. Practice deploying containers and managing them through the Docker CLI.\n   - Familiarize yourself with Terraform for infrastructure as code (IaC). Work through examples that require setting up resources.\n\n3. **Resolve Common Issues**:\n   - If you encounter any errors, like `ModuleNotFoundError`, check your installations. For example, you might need to install required packages such as `psycopg2`:\n     ```bash\n     pip install psycopg2-binary\n     ```\n   - Make sure to follow any specific error r

In [47]:

for entry in response.output:
    chat_messages.append(entry)
    print(entry.type)

    if entry.type == 'function_call':      
        result = do_call(entry)
        chat_messages.append(result)
    elif entry.type == 'message':
        print(entry.content[0].text) 

message
To excel in Module 1 of the course, consider the following strategies and tips:

1. **Understand the Core Concepts**:
   - Focus on the principles of Docker and Terraform since Module 1 primarily covers these technologies. Review any foundational materials provided in the course.

2. **Hands-On Practice**:
   - Set up a local environment using Docker. Practice deploying containers and managing them through the Docker CLI.
   - Familiarize yourself with Terraform for infrastructure as code (IaC). Work through examples that require setting up resources.

3. **Resolve Common Issues**:
   - If you encounter any errors, like `ModuleNotFoundError`, check your installations. For example, you might need to install required packages such as `psycopg2`:
     ```bash
     pip install psycopg2-binary
     ```
   - Make sure to follow any specific error resolution steps mentioned in the FAQ and community posts.

4. **Use Community Resources**:
   - Engage with your peers in discussion forum

In [48]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.
When using FAQ, perform deep topic exploration: make one request to FAQ,
and then based on the results, make more requests.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_messages = [
    {"role": "developer", "content": developer_prompt},
]

In [49]:
while True: # main Q&A loop
    question = input() # How do I do my best for module 1?
    if question == 'stop':
        break

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True: # request-response loop - query API till get a message
        response = client.responses.create(
            model='gpt-4o-mini',
            input=chat_messages,
            tools=tools
        )

        has_messages = False
        
        for entry in response.output:
            chat_messages.append(entry)
        
            if entry.type == 'function_call':      
                print('function_call:', entry)
                print()
                result = do_call(entry)
                chat_messages.append(result)
            elif entry.type == 'message':
                print(entry.content[0].text)
                print()
                has_messages = True

        if has_messages:
            break

function_call: ResponseFunctionToolCall(arguments='{"query":"module 1 best practices"}', call_id='call_Wmf43AnXd5LmSCSj2VEMWlH6', name='search', type='function_call', id='fc_6878637339b4819a923fa8f1a8e15fe3080df6a92742c352', status='completed')

function_call: ResponseFunctionToolCall(arguments='{"query":"module 1 best practices Docker Terraform"}', call_id='call_eLv2hR1wSPlO14aCmM19m9n4', name='search', type='function_call', id='fc_687863749f4c819ab5336bcc6d671b91080df6a92742c352', status='completed')

To excel in Module 1, which focuses on Docker and Terraform, here are some best practices and tips:

1. **Understand Docker Basics**:
   - Familiarize yourself with Docker concepts such as containers, images, and volumes. The Docker documentation provides best practices for optimizing your Docker setup.

2. **Performance Considerations**:
   - If you are using Windows, be aware that Docker runs on the WSL2 backend. Store all your code in your default Linux distro to maximize file system

### Multiple tools

In [50]:
def add_entry(question, answer):
    doc = {
        'question': question,
        'text': answer,
        'section': 'user added',
        'course': 'data-engineering-zoomcamp'
    }
    index.append(doc)

In [51]:
add_entry_description = {
    "type": "function",
    "name": "add_entry",
    "description": "Add an entry to the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question to be added to the FAQ database",
            },
            "answer": {
                "type": "string",
                "description": "The answer to the question",
            }
        },
        "required": ["question", "answer"],
        "additionalProperties": False
    }
}

In [52]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(search, search_tool)

tools.get_tools()

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [53]:
chat.run()

Chat ended.


In [54]:
tools.add_tool(add_entry, add_entry_description)
tools.get_tools()

[{'type': 'function',
  'name': 'search',
  'description': 'Search the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'Search query text to look up in the course FAQ.'}},
   'required': ['query'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'add_entry',
  'description': 'Add an entry to the FAQ database',
  'parameters': {'type': 'object',
   'properties': {'question': {'type': 'string',
     'description': 'The question to be added to the FAQ database'},
    'answer': {'type': 'string', 'description': 'The answer to the question'}},
   'required': ['question', 'answer'],
   'additionalProperties': False}}]

In [55]:
chat.run()

Chat ended.


In [56]:
index.docs[-1]

{'question': 'How do I do well in Module 1?',
 'text': "1. Understand the Basics: Familiarize yourself with Docker and Terraform concepts since they are fundamental to the module. Make sure you understand containerization and infrastructure as code.\n\n2. Follow Instructions Carefully: Ensure that you follow the course instructions carefully when setting up your environment. Misconfiguration can lead to errors.\n\n3. Practice Coding: Implement the examples given in the module. Hands-on practice with commands and codes related to Docker and Terraform will help reinforce your understanding.\n\n4. Troubleshoot Common Errors: Familiarize yourself with common errors, such as:\n   - `ModuleNotFoundError: No module named 'psycopg2'`: Install the module using `pip install psycopg2`.\n   - Errors related to SQLAlchemy: Ensure you're using the correct connection string format for your database.\n\n5. Engage with Fellow Students: Participate in discussions and study groups to share knowledge, cla